**Table of contents**<a id='toc0_'></a>    
- [ENSG](#toc1_)    
    - [How many total unique gene records are there in Ensembl](#toc1_1_1_)    
    - [Identify alias-alias collision symbols](#toc1_1_2_)    
    - [Create a histogram displaying how frequent the numebr of gene records sharing an alias is](#toc1_1_3_)    
- [HGNC](#toc2_)    
    - [How many total unique gene records are there in HGNC](#toc2_1_1_)    
    - [Identify alias-alias collision symbols](#toc2_1_2_)    
    - [Create a histogram displaying how frequent the numebr of gene records sharing an alias is](#toc2_1_3_)    
- [NCBI Info](#toc3_)    
    - [How many total unique gene records are there in NCBI Gene](#toc3_1_1_)    
    - [Identify alias-alias collision symbols](#toc3_1_2_)    
    - [Create a histogram displaying how frequent the numebr of gene records sharing an alias is](#toc3_1_3_)    
- [Merge to create Alias-Alias Collision Table- On Primary Gene Symbol](#toc4_)    
- [Merge to create Alias-Alias Collision Table- On Alias Symbol](#toc5_)    
- [How many unique primary gene symbols are there?](#toc6_)    
  - [Per Source](#toc6_1_)    
  - [All sources](#toc6_2_)    
    - [How many symbols appear in all sources?](#toc6_2_1_)    
    - [How many unique symbols are found between all sources?](#toc6_2_2_)    
- [How many unique aliases are there?](#toc7_)    
  - [Per Source](#toc7_1_)    
  - [All sources](#toc7_2_)    
    - [How many aliases appear in all sources?](#toc7_2_1_)    
    - [How many unique aliases are found between all sources?](#toc7_2_2_)    
- [How many gene records have an alias that is shared?](#toc8_)    
  - [Per Source](#toc8_1_)    
  - [All Sources](#toc8_2_)    
    - [How many gene records have at least one shared alias in all sources?](#toc8_2_1_)    
    - [How many unique gene records that have at least one shared alias are found between all sources?](#toc8_2_2_)    
- [How many alias symbols are being shared?](#toc9_)    
  - [Per Source](#toc9_1_)    
  - [All Sources](#toc9_2_)    
    - [How many aliases are shared in all sources?](#toc9_2_1_)    
    - [How many unique shared aliases are found between all sources?](#toc9_2_2_)    
- [How many gene concept-alias relationships are there?](#toc10_)    
  - [Per Source](#toc10_1_)    
  - [All Sources](#toc10_2_)    
    - [How many unique gene-alias pairs are found between all sources?](#toc10_2_1_)    
      - [Remove duplicate concept-alias pairs](#toc10_2_1_1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [762]:
import pandas as pd
import numpy as np
import plotly.express as px

In [763]:
def create_aa_collision_df(mini_xxxx_df: pd.DataFrame, source: str, split_on_character: str) -> pd.DataFrame:
    """Create a df of alias-alias collision symbols 

    :param mini_xxxx_df: Processed df of gene records
    :param source: Representation of the source of the gene records
    :param split_on_character: Character that is used to seperate alias symbols in the mini_xxxx_df
    :return: A df of genes that share an alias with another gene
    """

    #Drop genes without any aliases
    subset_genes_xxxx_df = mini_xxxx_df.replace(" ", np.nan)
    subset_genes_xxxx_df = subset_genes_xxxx_df.replace("", np.nan)
    subset_genes_xxxx_df = subset_genes_xxxx_df.replace("-", np.nan)
    subset_genes_xxxx_df = subset_genes_xxxx_df.dropna(subset=["alias_symbol"])

    #Rename alias_symbol to aliases
    subset_genes_xxxx_df = subset_genes_xxxx_df.rename(
    columns={"alias_symbol": "aliases"})

    #Make each value the alias symbol column a set
    subset_genes_xxxx_df["alias_symbol"] = subset_genes_xxxx_df["aliases"].astype(str)
    subset_genes_xxxx_df["alias_symbol"] = [x.split(split_on_character) for x in subset_genes_xxxx_df.alias_symbol]
    subset_genes_xxxx_df["alias_symbol"] = np.where(
    subset_genes_xxxx_df.alias_symbol == "", "", subset_genes_xxxx_df.alias_symbol.map(set)
    )

    #Explode the alias sets so that there is one alias per row
    subset_genes_xxxx_df = subset_genes_xxxx_df.explode("alias_symbol")
    
    #Remove duplicate instances of primary gene symbol- alias pairs
        #(occur because the same primary gene symbol may have multiple different ENSG IDs, 
        #see gene RFLNB for example)
    subset_genes_xxxx_df = subset_genes_xxxx_df.drop_duplicates(
    subset=["gene_symbol", "alias_symbol"], keep="first"
    )

    #Remove instances where the primary gene symbol is an alias for that gene record
        #the pair would be the same symbol twice
    subset_genes_xxxx_df = subset_genes_xxxx_df[subset_genes_xxxx_df["gene_symbol"] != subset_genes_xxxx_df["alias_symbol"]]

    #Convert the df into a csv and save
    subset_genes_xxxx_df.to_csv(f'../created_files/subset_genes_{source}_df.csv', index=True) 

    #Create df with genes that have an alias that can be found in another gene's alias set
    aa_collision_gene_xxxx_df = subset_genes_xxxx_df.copy()
    aa_collision_gene_xxxx_df["alias_duplicates"] = aa_collision_gene_xxxx_df.duplicated(
    subset="alias_symbol", keep=False
    )
    aa_collision_gene_xxxx_df = aa_collision_gene_xxxx_df[aa_collision_gene_xxxx_df["alias_duplicates"] == True]
    aa_collision_gene_xxxx_df = aa_collision_gene_xxxx_df.rename(
    columns={"alias_symbol": "collision_symbol"})
    aa_collision_gene_xxxx_df = aa_collision_gene_xxxx_df.drop(["alias_duplicates"], axis=1)
    aa_collision_gene_xxxx_df = aa_collision_gene_xxxx_df.sort_values("collision_symbol")

    #Add a source tag for future merging efforts
    aa_collision_gene_xxxx_df["source"] = str(source)

    #Convert the df into a csv
    aa_collision_gene_xxxx_df.to_csv(f'../created_files/aa_collision_gene_{source}_df.csv', index=True)

    #Create a secondary collision df that prioritizes the collision symbol
    aa_collision_alias_xxxx_df = aa_collision_gene_xxxx_df[
    ["collision_symbol", "gene_symbol", "aliases", "ENSG_ID", "source"]
    ]
    aa_collision_alias_xxxx_df = aa_collision_alias_xxxx_df.map(str)
    aa_collision_alias_xxxx_df = (
    aa_collision_alias_xxxx_df.groupby("collision_symbol")
    .agg({"ENSG_ID": ", ".join, "gene_symbol": ", ".join, "source": "first"})
    .reset_index()
    )

    #Convert the df into a csv and save
    aa_collision_alias_xxxx_df.to_csv(f'../created_files/aa_collision_alias_{source}_df.csv', index=True)

    return subset_genes_xxxx_df.head(), aa_collision_gene_xxxx_df.head(), aa_collision_alias_xxxx_df.head()

In [764]:
def create_aa_collision_histogram(aa_collision_gene_xxxx_df: pd.DataFrame, source: str, xxxx_alias_count: int):
    """Create a histogram of the frequencies at which aliases are shared

    :param aa_collision_gene_xxxx_df: A df of alias-alias collisions organized by primary gene symbol
    :param source: Representation of the source of the gene records
    :param xxxx_alias_count: Number of aliases total in the source
    :return: A histogram of the percentage of aliases that are shared between 2 genes, 3 genes, and so on
    """
    
    #Count the number of times each shared alias is used
    aa_collision_xxxx_count_df = aa_collision_gene_xxxx_df.pivot_table(
    index=["collision_symbol"], aggfunc="size"
    )
    aa_collision_xxxx_count_df = aa_collision_xxxx_count_df.reset_index()
    aa_collision_xxxx_count_df.rename(columns={0: "num_gene_records"}, inplace=True)
    aa_collision_xxxx_count_df = aa_collision_xxxx_count_df.sort_values(
        "num_gene_records", ascending=False)

    #Convert to csv
    aa_collision_xxxx_count_df.to_csv(f'../created_files/aa_collision_{source}_count_df.csv', index=True)

    #Count the frequency at which aliases are shared 
    aa_collision_xxxx_distribution_df = aa_collision_xxxx_count_df.pivot_table(
    index=["num_gene_records"], aggfunc="size"
    )
    aa_collision_xxxx_distribution_df = aa_collision_xxxx_distribution_df.reset_index()
    aa_collision_xxxx_distribution_df.rename(columns={0: "num_collision_symbol"}, inplace=True)
    aa_collision_xxxx_distribution_df["percent_collision_symbol"] = (
        aa_collision_xxxx_distribution_df["num_collision_symbol"] / xxxx_alias_count
    ) * 100

    #Convert to csv
    aa_collision_xxxx_distribution_df.to_csv(f'../created_files/aa_collision_{source}_distribution_df.csv', index=True)

    #Create histogram df 
    xxxx_alias_count_histogram_df = aa_collision_xxxx_distribution_df.drop(
    "num_collision_symbol", axis=1)

    #Convert to csv
    xxxx_alias_count_histogram_df.to_csv(f'../created_files/{source}_alias_count_histogram_df.csv', index=True)

    return px.bar(xxxx_alias_count_histogram_df, x="num_gene_records", y="percent_collision_symbol")



# <a id='toc1_'></a>[ENSG](#toc0_)

In [765]:
mini_ensg_df = pd.read_csv(
    "../created_files/mini_ensg_df.csv",
    dtype={"HGNC_ID": pd.Int64Dtype(), "NCBI_ID": pd.Int64Dtype()},
)
mini_ensg_df

,ENSG_ID,gene_symbol,HGNC_ID,NCBI_ID,alias_symbol
0,ENSG00000000003,TSPAN6,11858,7105,"T245, TM4SF6, TSPAN-6"
1,ENSG00000000005,TNMD,17757,64102,"BRICD4, CHM1L, MYODULIN, TEM, TENDIN"
2,ENSG00000000419,DPM1,3005,8813,"CDGIE, MPDS"
3,ENSG00000000457,SCYL3,19285,57147,"PACE-1, PACE1"
4,ENSG00000000460,FIRRM,25565,55732,"APOLO1, C1ORF112, FLIP, FLJ10706, MEICA1"
...,...,...,...,...,...
75829,ENSG00000293596,NaN,<NA>,105372654,NaN
75830,ENSG00000293597,LINC00970,48730,101978719,NaN
75831,ENSG00000293599,NaN,<NA>,<NA>,NaN
75832,ENSG00000293600,NaN,<NA>,131768270,NaN


### <a id='toc1_1_1_'></a>[How many total unique gene records are there in Ensembl](#toc0_)

By ENSG ID

In [766]:
ensg_gene_id_set = set(mini_ensg_df["ENSG_ID"])
len(ensg_gene_id_set)

70611

### <a id='toc1_1_2_'></a>[Identify alias-alias collision symbols](#toc0_)

In [767]:
create_aa_collision_df(mini_ensg_df, source="ENSG", split_on_character= ",")

(           ENSG_ID gene_symbol  HGNC_ID  NCBI_ID  \
 0  ENSG00000000003      TSPAN6    11858     7105   
 0  ENSG00000000003      TSPAN6    11858     7105   
 0  ENSG00000000003      TSPAN6    11858     7105   
 1  ENSG00000000005        TNMD    17757    64102   
 1  ENSG00000000005        TNMD    17757    64102   
 
                                 aliases alias_symbol  
 0                 T245, TM4SF6, TSPAN-6         T245  
 0                 T245, TM4SF6, TSPAN-6       TM4SF6  
 0                 T245, TM4SF6, TSPAN-6      TSPAN-6  
 1  BRICD4, CHM1L, MYODULIN, TEM, TENDIN     MYODULIN  
 1  BRICD4, CHM1L, MYODULIN, TEM, TENDIN          TEM  ,
                ENSG_ID gene_symbol  HGNC_ID  NCBI_ID  \
 8000   ENSG00000140379      BCL2A1      991      597   
 58193  ENSG00000275176       ACACA       84       31   
 1354   ENSG00000076555       ACACB       85       32   
 8000   ENSG00000140379      BCL2A1      991      597   
 2085   ENSG00000097021       ACOT7    24157    11332   
 

In [768]:
subset_genes_ensg_df = pd.read_csv(
    "../created_files/subset_genes_ensg_df.csv", index_col=[0])
subset_genes_ensg_df

,ENSG_ID,gene_symbol,HGNC_ID,NCBI_ID,aliases,alias_symbol
0,ENSG00000000003,TSPAN6,11858.0,7105.0,"T245, TM4SF6, TSPAN-6",T245
0,ENSG00000000003,TSPAN6,11858.0,7105.0,"T245, TM4SF6, TSPAN-6",TM4SF6
0,ENSG00000000003,TSPAN6,11858.0,7105.0,"T245, TM4SF6, TSPAN-6",TSPAN-6
1,ENSG00000000005,TNMD,17757.0,64102.0,"BRICD4, CHM1L, MYODULIN, TEM, TENDIN",MYODULIN
1,ENSG00000000005,TNMD,17757.0,64102.0,"BRICD4, CHM1L, MYODULIN, TEM, TENDIN",TEM
...,...,...,...,...,...,...
75744,ENSG00000293496,TMED11P,NaN,100379220.0,"p24a1, p24alpha1",p24alpha1
75744,ENSG00000293496,TMED11P,NaN,100379220.0,"p24a1, p24alpha1",p24a1
75752,ENSG00000293504,NPY6R,NaN,4888.0,"NPY1RL, NPY6RP, PP2, Y2B",Y2B
75790,ENSG00000293543,DUSP13A,56772.0,128854680.0,"BEDP, MDSP",MDSP


In [769]:
subset_genes_ensg_df.loc[subset_genes_ensg_df["gene_symbol"] == "NPY6R"]

,ENSG_ID,gene_symbol,HGNC_ID,NCBI_ID,aliases,alias_symbol
28798,ENSG00000226306,NPY6R,7959.0,NaN,"NPY1RL, NPY6RP, PP2",NPY1RL
28798,ENSG00000226306,NPY6R,7959.0,NaN,"NPY1RL, NPY6RP, PP2",NPY6RP
28798,ENSG00000226306,NPY6R,7959.0,NaN,"NPY1RL, NPY6RP, PP2",PP2
75752,ENSG00000293504,NPY6R,NaN,4888.0,"NPY1RL, NPY6RP, PP2, Y2B",Y2B


In [770]:
aa_collision_gene_ensg_df = pd.read_csv(
    "../created_files/aa_collision_gene_ensg_df.csv", index_col=[0])
aa_collision_gene_ensg_df

,ENSG_ID,gene_symbol,HGNC_ID,NCBI_ID,aliases,collision_symbol,source
8000,ENSG00000140379,BCL2A1,991.0,597.0,"ACC-1, ACC-2, ACC1, ACC2, BCL2L5, BFL1, GRS, H...",ACC1,ENSG
58193,ENSG00000275176,ACACA,84.0,31.0,"ACAC, ACAC1, ACACALPHA, ACC, ACC-ALPHA, ACC1, ...",ACC1,ENSG
1354,ENSG00000076555,ACACB,85.0,32.0,"ACACBETA, ACC-BETA, ACC2, ACCB, ACCBETA, HACC275",ACC2,ENSG
8000,ENSG00000140379,BCL2A1,991.0,597.0,"ACC-1, ACC-2, ACC1, ACC2, BCL2L5, BFL1, GRS, H...",ACC2,ENSG
2085,ENSG00000097021,ACOT7,24157.0,11332.0,"ACH1, ACT, BACH, CTE-II, HBACH, LACH1, MGC1126",ACT,ENSG
...,...,...,...,...,...,...,...
2565,ENSG00000101557,USP14,12612.0,9097.0,"TGT, UBP6",TGT,ENSG
6615,ENSG00000132388,UBE2G1,12482.0,7326.0,"UBC7, UBE2G",UBC7,ENSG
15955,ENSG00000184787,UBE2G2,12483.0,7327.0,UBC7,UBC7,ENSG
5350,ENSG00000122861,PLAU,9052.0,5328.0,"UPA, URK",UPA,ENSG


In [771]:
aa_collision_alias_ensg_df = pd.read_csv(
    "../created_files/aa_collision_alias_ensg_df.csv", index_col=[0])
aa_collision_alias_ensg_df

,collision_symbol,ENSG_ID,gene_symbol,source
0,ACC1,"ENSG00000140379, ENSG00000275176","BCL2A1, ACACA",ENSG
1,ACC2,"ENSG00000076555, ENSG00000140379","ACACB, BCL2A1",ENSG
2,ACT,"ENSG00000097021, ENSG00000196136","ACOT7, SERPINA3",ENSG
3,AGPAT9,"ENSG00000138678, ENSG00000153395","GPAT3, LPCAT1",ENSG
4,AIP1,"ENSG00000136848, ENSG00000187391","DAB2IP, MAGI2",ENSG
...,...,...,...,...
1144,TCRBV15S1,"ENSG00000276819, ENSG00000211750","TRBV15, TRBV24-1",ENSG
1145,TCRGV5P,"ENSG00000228668, ENSG00000226212","TRGV5P, TRGV6",ENSG
1146,TGT,"ENSG00000213339, ENSG00000101557","QTRT1, USP14",ENSG
1147,UBC7,"ENSG00000132388, ENSG00000184787","UBE2G1, UBE2G2",ENSG


### <a id='toc1_1_3_'></a>[Create a histogram displaying how frequent the numebr of gene records sharing an alias is](#toc0_)

In [772]:
ensg_alias_symbol_set = set(subset_genes_ensg_df["alias_symbol"])
ensg_alias_count = len(ensg_alias_symbol_set)

In [773]:
create_aa_collision_histogram(aa_collision_gene_ensg_df, "ENSG", ensg_alias_count)

In [774]:
aa_collision_ensg_count_df = pd.read_csv(
    "../created_files/aa_collision_ensg_count_df.csv", index_col=[0])

In [775]:
aa_collision_ensg_distribution_df = pd.read_csv(
    "../created_files/aa_collision_ensg_distribution_df.csv", index_col=[0])

In [776]:
ensg_alias_count_histogram_df = pd.read_csv(
    "../created_files/ensg_alias_count_histogram_df.csv", index_col=[0])

# <a id='toc2_'></a>[HGNC](#toc0_)

In [777]:
mini_hgnc_df = pd.read_csv(
    "../created_files/mini_hgnc_df.csv",
    dtype={"HGNC_ID": pd.Int64Dtype(), "NCBI_ID": pd.Int64Dtype()},
)
mini_hgnc_df

,ENSG_ID,gene_symbol,HGNC_ID,NCBI_ID,alias_symbol
0,ENSG00000000003,TSPAN6,11858,7105,"T245, TSPAN-6"
1,ENSG00000000005,TNMD,17757,64102,"myodulin, ChM1L, tendin, TEM, BRICD4"
2,ENSG00000000419,DPM1,3005,8813,"MPDS, CDGIE"
3,ENSG00000000457,SCYL3,19285,57147,"PACE-1, PACE1"
4,ENSG00000000460,FIRRM,25565,55732,"FLJ10706, Apolo1, FLIP, MEICA1"
...,...,...,...,...,...
45641,NaN,ZNF97,13173,<NA>,NaN
45642,NaN,ZNFP1,13181,<NA>,NaN
45643,NaN,ZPAXP,51635,105373450,ZPX1P
45644,NaN,ZRK,13193,<NA>,NaN


### <a id='toc2_1_1_'></a>[How many total unique gene records are there in HGNC](#toc0_)

By HGNC ID

In [778]:
hgnc_gene_id_set = set(mini_hgnc_df["HGNC_ID"])
len(hgnc_gene_id_set)

45646

### <a id='toc2_1_2_'></a>[Identify alias-alias collision symbols](#toc0_)

In [779]:
create_aa_collision_df(mini_hgnc_df, source="HGNC", split_on_character= ",")

(           ENSG_ID gene_symbol  HGNC_ID  NCBI_ID  \
 0  ENSG00000000003      TSPAN6    11858     7105   
 0  ENSG00000000003      TSPAN6    11858     7105   
 1  ENSG00000000005        TNMD    17757    64102   
 1  ENSG00000000005        TNMD    17757    64102   
 1  ENSG00000000005        TNMD    17757    64102   
 
                                 aliases alias_symbol  
 0                         T245, TSPAN-6         T245  
 0                         T245, TSPAN-6      TSPAN-6  
 1  myodulin, ChM1L, tendin, TEM, BRICD4     myodulin  
 1  myodulin, ChM1L, tendin, TEM, BRICD4          TEM  
 1  myodulin, ChM1L, tendin, TEM, BRICD4        ChM1L  ,
                ENSG_ID gene_symbol  HGNC_ID  NCBI_ID  \
 75     ENSG00000005022     SLC25A5    10991      292   
 7761   ENSG00000139187       KLRG1     6380    10219   
 8398   ENSG00000143546      S100A8    10498     6279   
 10916  ENSG00000163220      S100A9    10499     6280   
 9226   ENSG00000149735       GPHA2    18054   170589   
 

In [780]:
subset_genes_hgnc_df = pd.read_csv(
    "../created_files/subset_genes_hgnc_df.csv", index_col=[0])

In [781]:
subset_genes_hgnc_df.loc[subset_genes_hgnc_df["gene_symbol"] == "NPY6R"]

,ENSG_ID,gene_symbol,HGNC_ID,NCBI_ID,aliases,alias_symbol
24464,ENSG00000226306,NPY6R,7959,4888.0,"PP2, NPY1RL, NPY6RP",NPY1RL
24464,ENSG00000226306,NPY6R,7959,4888.0,"PP2, NPY1RL, NPY6RP",NPY6RP
24464,ENSG00000226306,NPY6R,7959,4888.0,"PP2, NPY1RL, NPY6RP",PP2


In [782]:
aa_collision_gene_hgnc_df = pd.read_csv(
    "../created_files/aa_collision_gene_hgnc_df.csv", index_col=[0])

In [783]:
aa_collision_alias_hgnc_df = pd.read_csv(
    "../created_files/aa_collision_alias_hgnc_df.csv", index_col=[0])

### <a id='toc2_1_3_'></a>[Create a histogram displaying how frequent the numebr of gene records sharing an alias is](#toc0_)

In [784]:
hgnc_alias_symbol_set = set(subset_genes_hgnc_df["alias_symbol"])
hgnc_alias_count = len(hgnc_alias_symbol_set)

In [785]:
create_aa_collision_histogram(aa_collision_gene_hgnc_df, "HGNC", hgnc_alias_count)

In [786]:
aa_collision_hgnc_count_df = pd.read_csv(
    "../created_files/aa_collision_hgnc_count_df.csv", index_col=[0])

In [787]:
aa_collision_hgnc_distribution_df = pd.read_csv(
    "../created_files/aa_collision_hgnc_distribution_df.csv", index_col=[0])

In [788]:
hgnc_alias_count_histogram_df = pd.read_csv(
    "../created_files/hgnc_alias_count_histogram_df.csv", index_col=[0])

# <a id='toc3_'></a>[NCBI Info](#toc0_)

In [789]:
mini_ncbi_df = pd.read_csv(
    "../created_files/mini_ncbi_df.csv",
    dtype={"HGNC_ID": pd.Int64Dtype(), "NCBI_ID": pd.Int64Dtype()},
)
mini_ncbi_df

,NCBI_ID,gene_symbol,alias_symbol,HGNC_ID,ENSG_ID
0,1,A1BG,A1B|ABG|GAB|HYST2477,5,ENSG00000121410
1,2,A2M,A2MD|CPAMD5|FWP007|S863-7,7,ENSG00000175899
2,3,A2MP1,A2MP,8,ENSG00000291190
3,9,NAT1,AAC1|MNAT|NAT-1|NATI,7645,ENSG00000171428
4,10,NAT2,AAC2|NAT-2|PNAT,7646,ENSG00000156006
...,...,...,...,...,...
193451,8923215,trnD,-,<NA>,NaN
193452,8923216,trnP,-,<NA>,NaN
193453,8923217,trnA,-,<NA>,NaN
193454,8923218,COX1,-,<NA>,NaN


### <a id='toc3_1_1_'></a>[How many total unique gene records are there in NCBI Gene](#toc0_)

By ENSG ID

In [790]:
ncbi_gene_id_set = set(mini_ncbi_df["ENSG_ID"])
len(ncbi_gene_id_set)

36803

### <a id='toc3_1_2_'></a>[Identify alias-alias collision symbols](#toc0_)

In [791]:
create_aa_collision_df(mini_ncbi_df, source="NCBI", split_on_character= "|")

(   NCBI_ID gene_symbol                    aliases  HGNC_ID          ENSG_ID  \
 0        1        A1BG       A1B|ABG|GAB|HYST2477        5  ENSG00000121410   
 0        1        A1BG       A1B|ABG|GAB|HYST2477        5  ENSG00000121410   
 0        1        A1BG       A1B|ABG|GAB|HYST2477        5  ENSG00000121410   
 0        1        A1BG       A1B|ABG|GAB|HYST2477        5  ENSG00000121410   
 1        2         A2M  A2MD|CPAMD5|FWP007|S863-7        7  ENSG00000175899   
 
   alias_symbol  
 0          A1B  
 0     HYST2477  
 0          ABG  
 0          GAB  
 1       CPAMD5  ,
       NCBI_ID gene_symbol                                            aliases  \
 4525     5728        PTEN  10q23del|BZS|CWS1|DEC|GLM2|MHAM|MMAC1|PTEN1|PT...   
 537       657      BMPR1A     10q23del|ACVRLK3|ALK-3|ALK3|BMPR-1A|CD292|SKR5   
 199       239      ALOX12                               12-LOX|12S-LOX|LOG12   
 205       246      ALOX15                       12-LOX|15-LOX|15-LOX-1|LOG15   
 245

In [792]:
subset_genes_ncbi_df = pd.read_csv(
    "../created_files/subset_genes_ncbi_df.csv", index_col=[0])

In [793]:
aa_collision_gene_ncbi_df = pd.read_csv(
    "../created_files/aa_collision_gene_ncbi_df.csv", index_col=[0])

In [794]:
aa_collision_alias_ncbi_df = pd.read_csv(
    "../created_files/aa_collision_alias_ncbi_df.csv", index_col=[0])

### <a id='toc3_1_3_'></a>[Create a histogram displaying how frequent the numebr of gene records sharing an alias is](#toc0_)

In [795]:
ncbi_alias_symbol_set = set(subset_genes_ncbi_df["alias_symbol"])
ncbi_alias_count = len(ncbi_alias_symbol_set)

In [796]:
create_aa_collision_histogram(aa_collision_gene_ncbi_df, "NCBI", ncbi_alias_count)

In [797]:
aa_collision_ncbi_count_df = pd.read_csv(
    "../created_files/aa_collision_ncbi_count_df.csv", index_col=[0])

In [798]:
aa_collision_ncbi_distribution_df = pd.read_csv(
    "../created_files/aa_collision_ncbi_distribution_df.csv", index_col=[0])

In [799]:
ncbi_alias_count_histogram_df = pd.read_csv(
    "../created_files/ncbi_alias_count_histogram_df.csv", index_col=[0])

In [800]:
len(subset_genes_ensg_df)

57354

In [801]:
len(subset_genes_hgnc_df)

44584

In [802]:
len(subset_genes_ncbi_df)

74053

# <a id='toc4_'></a>[Merge to create Alias-Alias Collision Table- On Primary Gene Symbol](#toc0_)

In [804]:
merged_aa_collision_gene_df = pd.concat(
    [
        aa_collision_gene_hgnc_df[["gene_symbol", "aliases", "ENSG_ID", "collision_symbol", "source"]],
        aa_collision_gene_ncbi_df[["gene_symbol", "aliases", "ENSG_ID", "collision_symbol", "source"]],
        aa_collision_gene_ensg_df[["gene_symbol", "aliases", "ENSG_ID", "collision_symbol", "source"]],
    ]
)
merged_aa_collision_gene_df

,gene_symbol,aliases,ENSG_ID,collision_symbol,source
75,SLC25A5,"T2, 2F1, T3",ENSG00000005022,2F1,HGNC
7761,KLRG1,"MAFA, 2F1, MAFA-L, CLEC15A",ENSG00000139187,2F1,HGNC
8398,S100A8,"P8, MRP8, MRP-8, 60B8AG, CGLA, S100-A8",ENSG00000143546,60B8AG,HGNC
10916,S100A9,"P14, MIF, NIF, LIAG, MRP14, MAC387, 60B8AG, CG...",ENSG00000163220,60B8AG,HGNC
9226,GPHA2,"GPA2, ZSIG51, A2, MGC126572",ENSG00000149735,A2,HGNC
...,...,...,...,...,...
2565,USP14,"TGT, UBP6",ENSG00000101557,TGT,ENSG
6615,UBE2G1,"UBC7, UBE2G",ENSG00000132388,UBC7,ENSG
15955,UBE2G2,UBC7,ENSG00000184787,UBC7,ENSG
5350,PLAU,"UPA, URK",ENSG00000122861,UPA,ENSG


In [805]:
merged_aa_collision_gene_df.to_csv(
    "../created_files/merged_aa_collision_gene_df.csv", index=False
)

In [807]:
merged_aa_collision_gene_df.loc[merged_aa_collision_gene_df.collision_symbol == "ALP"]

,gene_symbol,aliases,ENSG_ID,collision_symbol,source
9748,PDLIM3,ALP,ENSG00000154553,ALP,HGNC
21413,CCL27,"ALP, ILC, CTACK, skinkine, ESkine, PESKY, CTAK",ENSG00000213927,ALP,HGNC
41314,ATHS,ALP,NaN,ALP,HGNC
209,ALPP,ALP|ALPI|IAP|PALP|PLAP|PLAP-1,ENSG00000163283,ALP,NCBI
9843,ATRNL1,ALP|bA338L11.1|bA454H24.1,ENSG00000107518,ALP,NCBI
5223,SLPI,ALK1|ALP|BLPI|HUSI|HUSI-I|MPI|WAP4|WFDC4,ENSG00000124107,ALP,NCBI
15068,ASRGL1,ALP|ALP1|CRASH,ENSG00000162174,ALP,NCBI
8497,CCL27,ALP|CTACK|CTAK|ESKINE|ILC|PESKY|SCYA27,ENSG00000213927,ALP,NCBI
391,ATHS,ALP,NaN,ALP,NCBI
10410,PDLIM3,ALP,ENSG00000154553,ALP,NCBI


# <a id='toc5_'></a>[Merge to create Alias-Alias Collision Table- On Alias Symbol](#toc0_)

In [809]:
merged_aa_collision_alias_df = pd.concat(
    [
        aa_collision_alias_hgnc_df[["collision_symbol", "gene_symbol", "ENSG_ID", "source"]],
        aa_collision_alias_ncbi_df[["collision_symbol", "gene_symbol", "ENSG_ID", "source"]],
        aa_collision_alias_ensg_df[["collision_symbol", "gene_symbol", "ENSG_ID", "source"]],
    ]
)
merged_aa_collision_alias_df

,collision_symbol,gene_symbol,ENSG_ID,source
0,2F1,"SLC25A5, KLRG1","ENSG00000005022, ENSG00000139187",HGNC
1,60B8AG,"S100A8, S100A9","ENSG00000143546, ENSG00000163220",HGNC
2,A2,"GPHA2, CFAP410, SGCA","ENSG00000149735, ENSG00000160226, ENSG00000108823",HGNC
3,ACC2,"BCL2A1, ACACB","ENSG00000140379, ENSG00000076555",HGNC
4,ACS2,"ACSL6, ACSL5","ENSG00000164398, ENSG00000197142",HGNC
...,...,...,...,...
1144,TCRBV15S1,"TRBV15, TRBV24-1","ENSG00000276819, ENSG00000211750",ENSG
1145,TCRGV5P,"TRGV5P, TRGV6","ENSG00000228668, ENSG00000226212",ENSG
1146,TGT,"QTRT1, USP14","ENSG00000213339, ENSG00000101557",ENSG
1147,UBC7,"UBE2G1, UBE2G2","ENSG00000132388, ENSG00000184787",ENSG


In [810]:
merged_aa_collision_alias_df["gene_symbol"] = merged_aa_collision_alias_df[
    "gene_symbol"
].str.split(",")
merged_aa_collision_alias_df["gene_symbol_count"] = [
    len(c) for c in merged_aa_collision_alias_df["gene_symbol"]
]
merged_aa_collision_alias_df = merged_aa_collision_alias_df.sort_values(
    by="gene_symbol_count", ascending=False
)
merged_aa_collision_alias_df

,collision_symbol,gene_symbol,ENSG_ID,source,gene_symbol_count
3305,VH,"[IGHV4-4, IGHV3-7, IGHV3-66, IGHM, IGHV4-2...","ENSG00000276775, ENSG00000211938, ENSG00000211...",NCBI,36
1303,H4-16,"[H4C12, H4C5, H4C6, H4C1, H4C16, H4C15, ...","ENSG00000273542, ENSG00000276966, ENSG00000274...",NCBI,14
1317,H4C9,"[H4C2, H4C4, H4C3, H4C15, H4C12, H4C16, ...","ENSG00000278705, ENSG00000277157, ENSG00000197...",NCBI,13
1304,H4C1,"[H4C14, H4C12, H4C3, H4C16, H4C6, H4C2, ...","ENSG00000270882, ENSG00000273542, ENSG00000197...",NCBI,13
1305,H4C11,"[H4C1, H4C3, H4C5, H4C4, H4C8, H4C6, H4C...","ENSG00000278637, ENSG00000197061, ENSG00000276...",NCBI,13
...,...,...,...,...,...
1240,GST3,"[GSTP1, CHST4]","ENSG00000084207, ENSG00000140835",NCBI,2
1238,GST1,"[GSPT1, GSTM1]","ENSG00000103342, ENSG00000134184",NCBI,2
1237,GST,"[GSTK1, SLCO6A1]","ENSG00000197448, ENSG00000205359",NCBI,2
1236,GSP,"[GNAS, GSM1]","ENSG00000087460, nan",NCBI,2


In [812]:
merged_aa_collision_alias_df.loc[merged_aa_collision_alias_df["collision_symbol"] == "ASP"]

,collision_symbol,gene_symbol,ENSG_ID,source,gene_symbol_count
222,ASP,"[ASPA, ROPN1L, TMPRSS11D, ASPM, ATG5, C3,...","ENSG00000108381, ENSG00000145491, ENSG00000153...",NCBI,8
394,ASP,"[ASIP, ROPN1L, ATG5, ASPA]","ENSG00000101440, ENSG00000145491, ENSG00000057...",HGNC,4
864,ASP,"[TMPRSS11D, ASPM, ROPN1L]","ENSG00000153802, ENSG00000066279, ENSG00000145491",ENSG,3


In [813]:
merged_aa_collision_alias_df.to_csv(
    "../created_files/merged_aa_collision_alias_df.csv", index=True, quoting=0
)

In [815]:
aa_collision_set = set(merged_aa_collision_alias_df["collision_symbol"].tolist())
len(aa_collision_set)

4494

# <a id='toc6_'></a>[How many unique primary gene symbols are there?](#toc0_)

## <a id='toc6_1_'></a>[Per Source](#toc0_)

In [816]:
ensg_gene_symbol_set = set(mini_ensg_df["gene_symbol"])
ensg_gene_symbol_count = len(ensg_gene_symbol_set)

In [817]:
hgnc_gene_symbol_set = set(mini_hgnc_df["gene_symbol"])
hgnc_gene_symbol_count = len(hgnc_gene_symbol_set)

In [818]:
ncbi_gene_symbol_set = set(mini_ncbi_df["gene_symbol"])
ncbi_gene_symbol_count = len(ncbi_gene_symbol_set)

In [819]:
unique_primary_symbol_summary_index = "HGNC", "ENSG", "NCBI"
unique_primary_symbol_summary = {
    "Number of Unique Primary Gene Symbols": [
        ensg_gene_symbol_count,
        hgnc_gene_symbol_count,
        ncbi_gene_symbol_count,
    ]
}
unique_primary_symbol_summary_df = pd.DataFrame(
    unique_primary_symbol_summary, index = unique_primary_symbol_summary_index
)
unique_primary_symbol_summary_df

,Number of Unique Primary Gene Symbols
HGNC,41068
ENSG,45646
NCBI,193303


## <a id='toc6_2_'></a>[All sources](#toc0_)

### <a id='toc6_2_1_'></a>[How many symbols appear in all sources?](#toc0_)

In [820]:
all_sources_unique_primary_symbol_set = (
    ensg_gene_symbol_set
    & hgnc_gene_symbol_set
    & ncbi_gene_symbol_set
)
all_sources_unique_primary_symbol_count = len(all_sources_unique_primary_symbol_set)
all_sources_unique_primary_symbol_count

40885

### <a id='toc6_2_2_'></a>[How many unique symbols are found between all sources?](#toc0_)

In [821]:
bw_all_sources_unique_primary_symbol_df = pd.concat(
    [
        mini_ensg_df[["alias_symbol", "gene_symbol"]],
        mini_hgnc_df[["alias_symbol", "gene_symbol"]],
        mini_ncbi_df[["alias_symbol", "gene_symbol"]],
    ]
)

In [822]:
bw_all_sources_unique_primary_symbol_set = set(bw_all_sources_unique_primary_symbol_df["gene_symbol"])
bw_all_sources_unique_primary_symbol_count = len(bw_all_sources_unique_primary_symbol_set)
bw_all_sources_unique_primary_symbol_count

194866

# <a id='toc7_'></a>[How many unique aliases are there?](#toc0_)

## <a id='toc7_1_'></a>[Per Source](#toc0_)

In [823]:
unique_alias_summary_index = "HGNC", "ENSG", "NCBI"
unique_alias_summary = {
    "Number of Unique Aliases": [
        ensg_alias_count,
        hgnc_alias_count,
        ncbi_alias_count,
    ]
}
unique_alias_summary_df = pd.DataFrame(
    unique_alias_summary, index = unique_alias_summary_index
)
unique_alias_summary_df

,Number of Unique Aliases
HGNC,55930
ENSG,43770
NCBI,69156


## <a id='toc7_2_'></a>[All sources](#toc0_)

### <a id='toc7_2_1_'></a>[How many aliases appear in all sources?](#toc0_)

In [824]:
all_sources_unique_alias_set = (
    ensg_alias_symbol_set
    & hgnc_alias_symbol_set
    & ncbi_alias_symbol_set
)
all_sources_unique_alias_count = len(all_sources_unique_alias_set)
all_sources_unique_alias_count

5574

### <a id='toc7_2_2_'></a>[How many unique aliases are found between all sources?](#toc0_)

In [825]:
bw_all_sources_unique_alias_df = pd.concat(
    [
        subset_genes_ensg_df[["alias_symbol", "gene_symbol"]],
        subset_genes_hgnc_df[["alias_symbol", "gene_symbol"]],
        subset_genes_ncbi_df[["alias_symbol", "gene_symbol"]],
    ]
)

In [826]:
bw_all_sources_unique_alias_set = set(bw_all_sources_unique_alias_df["alias_symbol"])
bw_all_sources_unique_alias_count = len(bw_all_sources_unique_alias_set)
bw_all_sources_unique_alias_count

125371

# <a id='toc8_'></a>[How many gene records have an alias that is shared?](#toc0_)

## <a id='toc8_1_'></a>[Per Source](#toc0_)

In [827]:
ensg_aa_collision_primary_symbol_set = set(aa_collision_gene_ensg_df["gene_symbol"])
ensg_aa_collision_primary_symbol_count = len(ensg_aa_collision_primary_symbol_set)

In [828]:
hgnc_aa_collision_primary_symbol_set = set(aa_collision_gene_hgnc_df["gene_symbol"])
hgnc_aa_collision_primary_symbol_count = len(hgnc_aa_collision_primary_symbol_set)

In [829]:
ncbi_aa_collision_primary_symbol_set = set(aa_collision_gene_ncbi_df["gene_symbol"])
ncbi_aa_collision_primary_symbol_count = len(ncbi_aa_collision_primary_symbol_set)

In [830]:
aa_collision_primary_symbol_summary_index = "HGNC", "ENSG", "NCBI"
aa_collision_primary_symbol_summary= {
    "Number of Gene Records With a Shared Alias": [
        ensg_aa_collision_primary_symbol_count,
        hgnc_aa_collision_primary_symbol_count,
        ncbi_aa_collision_primary_symbol_count,
    ]
}
aa_collision_primary_symbol_summary_df = pd.DataFrame(
    aa_collision_primary_symbol_summary, index = aa_collision_primary_symbol_summary_index
)
aa_collision_primary_symbol_summary_df

,Number of Gene Records With a Shared Alias
HGNC,2224
ENSG,1356
NCBI,5732


## <a id='toc8_2_'></a>[All Sources](#toc0_)

### <a id='toc8_2_1_'></a>[How many gene records have at least one shared alias in all sources?](#toc0_)

In [831]:
all_sources_aa_collision_genes = (
    ensg_aa_collision_primary_symbol_set
    & hgnc_aa_collision_primary_symbol_set
    & ncbi_aa_collision_primary_symbol_set
)
len(all_sources_aa_collision_genes)

995

### <a id='toc8_2_2_'></a>[How many unique gene records that have at least one shared alias are found between all sources?](#toc0_)

In [833]:
bw_all_sources_aa_collision_df = pd.concat(
    [
        aa_collision_gene_ensg_df[["collision_symbol", "gene_symbol"]],
        aa_collision_gene_hgnc_df[["collision_symbol", "gene_symbol"]],
        aa_collision_gene_ncbi_df[["collision_symbol", "gene_symbol"]],
    ]
)

In [834]:
bw_all_sources_aa_collision_genes_set = set(bw_all_sources_aa_collision_df["gene_symbol"])
bw_all_sources_aa_collision_genes_count = len(bw_all_sources_aa_collision_genes_set)
bw_all_sources_aa_collision_genes_count

5977

# <a id='toc9_'></a>[How many alias symbols are being shared?](#toc0_)

## <a id='toc9_1_'></a>[Per Source](#toc0_)

In [836]:
ensg_aa_collision_set = set(aa_collision_gene_ensg_df["collision_symbol"])
ensg_aa_collision_count = len(ensg_aa_collision_set)

In [838]:
hgnc_aa_collision_set = set(aa_collision_gene_hgnc_df["collision_symbol"])
hgnc_aa_collision_count = len(hgnc_aa_collision_set)

In [840]:
ncbi_aa_collision_set = set(aa_collision_gene_ncbi_df["collision_symbol"])
ncbi_aa_collision_count = len(ncbi_aa_collision_set)

In [841]:
aa_collision_alias_symbol_summary_index = "HGNC", "ENSG", "NCBI"
aa_collision_alias_symbol_summary = {
    "Number of Shared Aliases": [
        ensg_aa_collision_count,
        hgnc_aa_collision_count,
        ncbi_aa_collision_count,
    ]
}
aa_collision_alias_symbol_summary_df = pd.DataFrame(
    aa_collision_alias_symbol_summary, index = aa_collision_alias_symbol_summary_index
)
aa_collision_alias_symbol_summary_df

,Number of Shared Aliases
HGNC,1149
ENSG,673
NCBI,3476


## <a id='toc9_2_'></a>[All Sources](#toc0_)

### <a id='toc9_2_1_'></a>[How many aliases are shared in all sources?](#toc0_)

In [842]:
all_sources_aa_collision_aliases = (
    ensg_aa_collision_set
    & hgnc_aa_collision_set
    & ncbi_aa_collision_set
)
len(all_sources_aa_collision_aliases)

95

### <a id='toc9_2_2_'></a>[How many unique shared aliases are found between all sources?](#toc0_)

In [844]:
bw_all_sources_aa_collision_aliases_set = set(bw_all_sources_aa_collision_df["collision_symbol"])
bw_all_sources_aa_collision_aliases_count = len(bw_all_sources_aa_collision_aliases_set)
bw_all_sources_aa_collision_aliases_count

4494

# <a id='toc10_'></a>[How many gene concept-alias relationships are there?](#toc0_)

## <a id='toc10_1_'></a>[Per Source](#toc0_)

In [845]:
ensg_primary_alias_pair_count = len(subset_genes_ensg_df)

In [846]:
hgnc_primary_alias_pair_count = len(subset_genes_hgnc_df)

In [847]:
ncbi_primary_alias_pair_count = len(subset_genes_ncbi_df)

In [848]:
primary_alias_pairs_summary_index = "HGNC", "ENSG", "NCBI"
primary_alias_pairs_summary = {
    "Number of Unique Gene Concept-Alias Pairs": [
        ensg_primary_alias_pair_count,
        hgnc_primary_alias_pair_count,
        ncbi_primary_alias_pair_count,
    ]
}
primary_alias_pairs_summary_df = pd.DataFrame(
    primary_alias_pairs_summary, index=primary_alias_pairs_summary_index
)
primary_alias_pairs_summary_df

,Number of Unique Gene Concept-Alias Pairs
HGNC,57354
ENSG,44584
NCBI,74053


## <a id='toc10_2_'></a>[All Sources](#toc0_)

### <a id='toc10_2_1_'></a>[How many unique gene-alias pairs are found between all sources?](#toc0_)

In [849]:
bw_all_sources_primary_alias_pairs_df = pd.concat(
    [
        subset_genes_ensg_df[["alias_symbol", "gene_symbol"]],
        subset_genes_hgnc_df[["alias_symbol", "gene_symbol"]],
        subset_genes_ncbi_df[["alias_symbol", "gene_symbol"]],
    ]
)

In [850]:
len(bw_all_sources_primary_alias_pairs_df)

175991

#### <a id='toc10_2_1_1_'></a>[Remove duplicate concept-alias pairs](#toc0_)

In [851]:
bw_all_sources_primary_alias_pairs_df = bw_all_sources_primary_alias_pairs_df.drop_duplicates(
    subset=["gene_symbol", "alias_symbol"], keep="first"
)

In [852]:
len(bw_all_sources_primary_alias_pairs_df)

131856